In [53]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.metrics import roc_auc_score

# Reading datasets

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submn = pd.read_csv('submission.csv')

In [55]:
train

,customer_id,firstname,lastname,record_number,days_since_opened,days_since_confirmed,primary_term,final_term,days_till_primary_close,days_till_final_close,...,encoded_payment_21,encoded_payment_22,encoded_payment_23,encoded_payment_24,encoded_loans_account_holder_type,encoded_loans_credit_status,encoded_loans_credit_type,encoded_loans_account_currency,primary_close_flag,final_close_flag
0,4814dfa2-45a8-46b9-8102-98ecbbec2d2a,Navya,Vasa,7,10,8,12,10,13,5,...,0.0,1.0,3.0,4.0,1,3,4,1,0,0
1,bb0abe41-cc89-4a1d-bb1c-48bd212ae00d,Azad,Warrior,5,15,6,7,6,9,1,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
2,edfd1a62-05da-4cfd-b8ae-521f05f0fd5f,Piya,Jha,1,15,17,8,5,9,1,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
3,3eaa4c2a-2316-48f4-b1b0-4aafe2286630,Rhea,Varma,11,7,2,11,13,14,8,...,3.0,3.0,3.0,4.0,1,2,4,1,0,0
4,c9a73239-baae-40eb-8855-dab07767ea86,Sahil,Dave,12,14,9,8,11,6,13,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106669,c341a4e3-b4e2-455b-80b2-2be2655e6e85,Gatik,Boase,19,1,5,8,11,2,6,...,3.0,3.0,3.0,4.0,1,3,5,1,0,0
1106670,29d8337c-13c9-4bb6-b794-762c386b1866,Jhanvi,Andra,10,5,8,4,3,5,7,...,3.0,3.0,3.0,4.0,1,3,5,1,0,0
1106671,e3b374f2-d96e-4c8b-901a-be80a8bea3cc,Lakshit,Kulkarni,3,8,7,13,0,4,9,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
1106672,b81e5b47-5c1b-4302-897c-2edc179d583f,Parinaaz,Chaudry,4,10,4,16,9,6,13,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0


In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106674 entries, 0 to 1106673
Data columns (total 62 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   customer_id                         1106674 non-null  object 
 1   firstname                           1106674 non-null  object 
 2   lastname                            1106674 non-null  object 
 3   record_number                       1106674 non-null  int64  
 4   days_since_opened                   1106674 non-null  int64  
 5   days_since_confirmed                1106674 non-null  int64  
 6   primary_term                        1106674 non-null  int64  
 7   final_term                          1106674 non-null  int64  
 8   days_till_primary_close             1106674 non-null  int64  
 9   days_till_final_close               1106674 non-null  int64  
 10  loans_credit_limit                  1106674 non-null  int64  
 11  loans_next_

In [57]:
train.describe()

,record_number,days_since_opened,days_since_confirmed,primary_term,final_term,days_till_primary_close,days_till_final_close,loans_credit_limit,loans_next_payment_summary,loans_outstanding_balance,...,encoded_payment_21,encoded_payment_22,encoded_payment_23,encoded_payment_24,encoded_loans_account_holder_type,encoded_loans_credit_status,encoded_loans_credit_type,encoded_loans_account_currency,primary_close_flag,final_close_flag
count,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,...,1.069126e+06,1.069519e+06,1.070131e+06,1.070629e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06,1.106674e+06
mean,9.149727e+00,9.433967e+00,8.231377e+00,8.408147e+00,8.282763e+00,7.273099e+00,8.193988e+00,9.715922e+00,2.332612e+00,2.993031e+00,...,2.328069e+00,2.359353e+00,2.393023e+00,3.529236e+00,1.029407e+00,2.758336e+00,3.599215e+00,1.001480e+00,1.282708e-01,1.910454e-01
std,5.746106e+00,5.747728e+00,4.776280e+00,5.282007e+00,4.565543e+00,5.094787e+00,4.321941e+00,5.844795e+00,1.247430e+00,6.411320e-01,...,1.238293e+00,1.217475e+00,1.193804e+00,1.081633e+00,2.931483e-01,4.686307e-01,1.011067e+00,4.470347e-02,3.343913e-01,3.931249e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,5.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00,2.000000e+00,5.000000e+00,5.000000e+00,2.000000e+00,3.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,9.000000e+00,1.000000e+01,9.000000e+00,8.000000e+00,8.000000e+00,7.000000e+00,9.000000e+00,1.000000e+01,2.000000e+00,3.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,1.300000e+01,1.400000e+01,1.200000e+01,1.400000e+01,1.200000e+01,1.200000e+01,1.100000e+01,1.500000e+01,2.000000e+00,3.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,4.700000e+01,1.900000e+01,1.700000e+01,1.700000e+01,1.600000e+01,1.600000e+01,1.500000e+01,1.900000e+01,6.000000e+00,5.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,6.000000e+00,6.000000e+00,7.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00


In [58]:
#Customer
train[['customer_id', 'firstname', 'lastname']].value_counts()*100/train.shape[0]

customer_id                           firstname  lastname    
b9bb833c-d392-4a03-8eec-e4d9f62478b5  Yuvaan     Kothari         0.003163
bd3685fc-3999-46bd-9010-efa26e442c1b  Kabir      Balan           0.003163
c97302f6-afd5-40f4-bd43-fea6441de87f  Parinaaz   Luthra          0.003163
3a512b48-6900-48a8-bef3-0abcbcd921ed  Madhav     Uppal           0.003072
e5ea197c-fd71-412d-b520-66a7bfd32402  Vidur      Grover          0.003072
                                                                   ...   
9915dc5e-9ff3-46ff-bd30-aef7157fd699  Badal      Ramachandran    0.000181
7b7d322b-f461-4d94-97e5-233967dced49  Dishani    Sami            0.000181
ae430ad5-e578-4715-802e-287a3c61a9d4  Ehsaan     Kar             0.000181
9813f868-cb5b-4f08-ab95-c4ad6a5b37df  Umang      Srinivasan      0.000181
30c5117d-f43e-4b60-aa5f-4f6fac231787  Priyansh   Gupta           0.000090
Name: count, Length: 98376, dtype: float64

# Deriving new feature Customer_tag

In [59]:
#Lets encode into customer types
train['customer_tag'] = train['customer_id']+ '_' + train['firstname'].str.strip() + '_' + train['lastname'].str.strip()
train.drop(columns=['customer_id', 'firstname', 'lastname'], inplace=True)

test['customer_tag'] = test['customer_id']+ '_' + test['firstname'].str.strip() + '_' + test['lastname'].str.strip()
test.drop(columns=['customer_id', 'firstname', 'lastname'], inplace=True)

In [ ]:
# pip install pandas-profiling

In [ ]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(train)
# profile

# Encoding of boolean features

In [65]:
#Treat record_number as categorical feature
#Treat is_zero_loans_within_5_days to is_zero_loans_over_90_days as categorical feature
#Treat is_zero_utilization, is_zero_over_limit_count, is_zero_max_over_limit_count as categorical features
# cat_feat = train.select_dtypes(include='object').columns.tolist()

# cat_feat = ['record_number', 'customer_tag']
cat_feat = ['customer_tag']
boolean_feat = ['is_zero_loans_within_5_days',
 'is_zero_loans_within_5_to_30_days',
 'is_zero_loans_within_30_to_60_days',
 'is_zero_loans_within_60_to_90_days',
 'is_zero_loans_over_90_days',
 'is_zero_utilization',
 'is_zero_over_limit_count',
 'is_zero_max_over_limit_count']



for i in boolean_feat:
    train[i] = train[i].replace({'Yes': 1, 'No': 0})
    test[i] = test[i].replace({'Yes': 1, 'No': 0})
le = LabelEncoder()

full = pd.concat([train, test], axis=0, ignore_index=True)
full['customer_tag'] = le.fit_transform(full['customer_tag'])
train = full.iloc[:train.shape[0], ]
test = full.iloc[train.shape[0]:,]


In [10]:
test

,record_number,days_since_opened,days_since_confirmed,primary_term,final_term,days_till_primary_close,days_till_final_close,loans_credit_limit,loans_next_payment_summary,loans_outstanding_balance,...,encoded_payment_22,encoded_payment_23,encoded_payment_24,encoded_loans_account_holder_type,encoded_loans_credit_status,encoded_loans_credit_type,encoded_loans_account_currency,primary_close_flag,final_close_flag,customer_tag
1106674,14,2,13,14,2,14,6,1,2,3,...,3.0,3.0,NaN,1,3,4,1,NaN,NaN,11562
1106675,17,3,6,0,1,0,4,8,2,4,...,0.0,0.0,1.0,1,2,3,1,NaN,NaN,79723
1106676,14,9,10,14,7,10,4,2,5,1,...,3.0,3.0,4.0,1,2,4,1,NaN,NaN,28300
1106677,7,14,4,12,11,6,13,11,2,3,...,3.0,3.0,4.0,1,3,4,1,NaN,NaN,84086
1106678,13,3,3,16,1,15,4,16,2,3,...,3.0,3.0,4.0,1,3,3,1,NaN,NaN,60343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580958,1,10,4,7,11,6,13,5,6,3,...,3.0,3.0,4.0,1,3,4,1,NaN,NaN,79120
1580959,8,3,11,7,3,11,12,13,2,3,...,3.0,3.0,4.0,1,3,5,1,NaN,NaN,33093
1580960,12,6,13,12,10,16,7,16,2,3,...,0.0,1.0,1.0,1,3,4,1,NaN,NaN,32842
1580961,16,2,16,0,7,10,8,8,2,3,...,3.0,3.0,4.0,1,2,3,1,NaN,NaN,15021


In [66]:
train

,record_number,days_since_opened,days_since_confirmed,primary_term,final_term,days_till_primary_close,days_till_final_close,loans_credit_limit,loans_next_payment_summary,loans_outstanding_balance,...,encoded_payment_22,encoded_payment_23,encoded_payment_24,encoded_loans_account_holder_type,encoded_loans_credit_status,encoded_loans_credit_type,encoded_loans_account_currency,primary_close_flag,final_close_flag,customer_tag
0,7,10,8,12,10,13,5,11,2,3,...,1.0,3.0,4.0,1,3,4,1,0.0,0.0,27646
1,5,15,6,7,6,9,1,5,6,3,...,3.0,3.0,4.0,1,3,4,1,0.0,0.0,71862
2,1,15,17,8,5,9,1,6,2,3,...,3.0,3.0,4.0,1,3,4,1,0.0,0.0,91496
3,11,7,2,11,13,14,8,14,3,5,...,3.0,3.0,4.0,1,2,4,1,0.0,0.0,24007
4,12,14,9,8,11,6,13,5,6,3,...,3.0,3.0,4.0,1,3,4,1,0.0,0.0,77560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106669,19,1,5,8,11,2,6,18,2,3,...,3.0,3.0,4.0,1,3,5,1,0.0,0.0,75027
1106670,10,5,8,4,3,5,7,18,2,3,...,3.0,3.0,4.0,1,3,5,1,0.0,0.0,16174
1106671,3,8,7,13,0,4,9,3,2,3,...,3.0,3.0,4.0,1,3,4,1,0.0,0.0,87563
1106672,4,10,4,16,9,6,13,3,2,3,...,3.0,3.0,4.0,1,3,4,1,0.0,0.0,70738


In [ ]:
full

In [ ]:
train.info()

In [68]:
encoded_cols = train.columns.tolist()[28:53]
train[encoded_cols].describe()

,encoded_payment_0,encoded_payment_1,encoded_payment_2,encoded_payment_3,encoded_payment_4,encoded_payment_5,encoded_payment_6,encoded_payment_7,encoded_payment_8,encoded_payment_9,...,encoded_payment_15,encoded_payment_16,encoded_payment_17,encoded_payment_18,encoded_payment_19,encoded_payment_20,encoded_payment_21,encoded_payment_22,encoded_payment_23,encoded_payment_24
count,1.069055e+06,1.070930e+06,1.072062e+06,1.069258e+06,1.070834e+06,1.071370e+06,1.068655e+06,1.071045e+06,1.071912e+06,1.070823e+06,...,1.070000e+06,1.071962e+06,1.069359e+06,1.071406e+06,1.070293e+06,1.069155e+06,1.069126e+06,1.069519e+06,1.070131e+06,1.070629e+06
mean,1.612985e-01,3.855079e-01,5.334990e-01,6.637565e-01,7.978557e-01,9.307690e-01,1.093367e+00,1.251324e+00,1.362826e+00,1.472394e+00,...,2.092179e+00,2.136375e+00,2.177469e+00,2.221507e+00,2.261796e+00,3.296505e+00,2.328069e+00,2.359353e+00,2.393023e+00,3.529236e+00
std,6.233905e-01,9.388029e-01,1.096599e+00,1.201604e+00,1.287451e+00,1.354189e+00,1.414439e+00,1.452645e+00,1.469558e+00,1.477541e+00,...,1.363240e+00,1.344013e+00,1.324299e+00,1.301577e+00,1.279064e+00,1.258253e+00,1.238293e+00,1.217475e+00,1.193804e+00,1.081633e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00
max,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,...,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00


In [69]:
train[encoded_cols]

,encoded_payment_0,encoded_payment_1,encoded_payment_2,encoded_payment_3,encoded_payment_4,encoded_payment_5,encoded_payment_6,encoded_payment_7,encoded_payment_8,encoded_payment_9,...,encoded_payment_15,encoded_payment_16,encoded_payment_17,encoded_payment_18,encoded_payment_19,encoded_payment_20,encoded_payment_21,encoded_payment_22,encoded_payment_23,encoded_payment_24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,4.0
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0
2,0.0,NaN,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106669,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0
1106670,NaN,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,NaN,3.0,4.0,3.0,3.0,3.0,4.0
1106671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0
1106672,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,3.0,NaN,3.0,4.0,3.0,3.0,3.0,4.0


# Imputation of missing value

In [ ]:
#Fill nan with -1
train[encoded_cols] = train[encoded_cols].fillna(-1)
test[encoded_cols] = test[encoded_cols].fillna(-1)

In [71]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106674 entries, 0 to 1106673
Data columns (total 60 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   record_number                       1106674 non-null  int64  
 1   days_since_opened                   1106674 non-null  int64  
 2   days_since_confirmed                1106674 non-null  int64  
 3   primary_term                        1106674 non-null  int64  
 4   final_term                          1106674 non-null  int64  
 5   days_till_primary_close             1106674 non-null  int64  
 6   days_till_final_close               1106674 non-null  int64  
 7   loans_credit_limit                  1106674 non-null  int64  
 8   loans_next_payment_summary          1106674 non-null  int64  
 9   loans_outstanding_balance           1106674 non-null  int64  
 10  loans_max_overdue_amount            1106674 non-null  int64  
 11  loans_credi

In [72]:
encoded_loan_cols = train.columns.tolist()[53:57]
encoded_loan_cols

['encoded_loans_account_holder_type',
 'encoded_loans_credit_status',
 'encoded_loans_credit_type',
 'encoded_loans_account_currency']

In [74]:
train.columns.tolist()

['record_number',
 'days_since_opened',
 'days_since_confirmed',
 'primary_term',
 'final_term',
 'days_till_primary_close',
 'days_till_final_close',
 'loans_credit_limit',
 'loans_next_payment_summary',
 'loans_outstanding_balance',
 'loans_max_overdue_amount',
 'loans_credit_cost_rate',
 'loans_within_5_days',
 'loans_within_5_to_30_days',
 'loans_within_30_to_60_days',
 'loans_within_60_to_90_days',
 'loans_over_90_days',
 'is_zero_loans_within_5_days',
 'is_zero_loans_within_5_to_30_days',
 'is_zero_loans_within_30_to_60_days',
 'is_zero_loans_within_60_to_90_days',
 'is_zero_loans_over_90_days',
 'utilization',
 'over_limit_count',
 'max_over_limit_count',
 'is_zero_utilization',
 'is_zero_over_limit_count',
 'is_zero_max_over_limit_count',
 'encoded_payment_0',
 'encoded_payment_1',
 'encoded_payment_2',
 'encoded_payment_3',
 'encoded_payment_4',
 'encoded_payment_5',
 'encoded_payment_6',
 'encoded_payment_7',
 'encoded_payment_8',
 'encoded_payment_9',
 'encoded_payment_10',


# Model training

In [75]:
skf = StratifiedKFold(n_splits = 7, shuffle = True, random_state = 140)

input_cols = ['days_since_opened','days_since_confirmed','primary_term','final_term',
 'days_till_primary_close','days_till_final_close','loans_credit_limit','loans_next_payment_summary',
 'loans_outstanding_balance','loans_max_overdue_amount','loans_credit_cost_rate','loans_within_5_days',
 'loans_within_5_to_30_days','loans_within_30_to_60_days','loans_within_60_to_90_days','loans_over_90_days',
 'is_zero_loans_within_5_days','is_zero_loans_within_5_to_30_days','is_zero_loans_within_30_to_60_days',
 'is_zero_loans_within_60_to_90_days','is_zero_loans_over_90_days','utilization','over_limit_count',
 'max_over_limit_count','is_zero_utilization','is_zero_over_limit_count','is_zero_max_over_limit_count',
 'encoded_payment_0', 'encoded_payment_1','encoded_payment_2','encoded_payment_3','encoded_payment_4',
 'encoded_payment_5','encoded_payment_6','encoded_payment_7','encoded_payment_8','encoded_payment_9',
 'encoded_payment_10','encoded_payment_11','encoded_payment_12','encoded_payment_13','encoded_payment_14',
 'encoded_payment_15','encoded_payment_16','encoded_payment_17','encoded_payment_18','encoded_payment_19',
 'encoded_payment_20','encoded_payment_21','encoded_payment_22','encoded_payment_23','encoded_payment_24',
 'encoded_loans_account_holder_type','encoded_loans_credit_status','encoded_loans_credit_type','encoded_loans_account_currency']

output_cols = ['primary_close_flag','final_close_flag']

cat_feats = ['encoded_payment_0','encoded_payment_1','encoded_payment_2','encoded_payment_3','encoded_payment_4',
 'encoded_payment_5','encoded_payment_6','encoded_payment_7','encoded_payment_8','encoded_payment_9','encoded_payment_10',
 'encoded_payment_11','encoded_payment_12','encoded_payment_13','encoded_payment_14','encoded_payment_15','encoded_payment_16',
 'encoded_payment_17','encoded_payment_18','encoded_payment_19','encoded_payment_20','encoded_payment_21','encoded_payment_22',
'encoded_payment_23','encoded_payment_24','encoded_loans_account_holder_type','encoded_loans_credit_status','encoded_loans_credit_type',
 'encoded_loans_account_currency','is_zero_utilization','is_zero_over_limit_count','is_zero_max_over_limit_count','is_zero_loans_within_5_days',
 'is_zero_loans_within_5_to_30_days','is_zero_loans_within_30_to_60_days','is_zero_loans_within_60_to_90_days','is_zero_loans_over_90_days']

i=1
X = train[input_cols]
y1 = train[output_cols[0]]
y2 = train[output_cols[1]]
y1_cv_scores = []
y1_test_preds = []
y2_cv_scores = []
y2_test_preds = []
for train_index, test_index in skf.split(X, y1):
    print('{} of KFold {}'.format(i, skf.n_splits))
    xtr, xval = X.iloc[train_index], X.iloc[test_index]
    ytr, yval = y1.iloc[train_index], y1.iloc[test_index]

    model = XGBClassifier(random_state=42, n_jobs=-1, scale_pos_weight = y1[y1==0].shape[0]/y1[y1==1].shape[0], categorical_feature = cat_feats)
    model.fit(xtr, ytr)
    score = roc_auc_score(yval, model.predict(xval))
    print("ROC-AUC Score = ", score)
    y1_cv_scores.append(score)
    i+=1
    pred_test = model.predict(test[inp_cols])
    y1_test_preds.append(pred_test)

i=1
for train_index, test_index in skf.split(X, y2):
    print('{} of KFold {}'.format(i, skf.n_splits))
    xtr, xval = X.iloc[train_index], X.iloc[test_index]
    ytr, yval = y2.iloc[train_index], y2.iloc[test_index]

    model = XGBClassifier(random_state=42, n_jobs=-1, scale_pos_weight = y2[y2==0].shape[0]/y2[y2==1].shape[0], categorical_feature = cat_feats)
    model.fit(xtr, ytr)
    score = roc_auc_score(yval, model.predict(xval))
    print("ROC-AUC Score = ", score)
    y2_cv_scores.append(score)
    i+=1
    pred_test = model.predict(test[inp_cols])
    y2_test_preds.append(pred_test)
    

1 of KFold 7


/home/unmesh/anaconda3/envs/food-torch-env/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [17:07:41] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "categorical_feature" } are not used.

  warnings.warn(smsg, UserWarning)


ROC-AUC Score =  0.9998157839799885
2 of KFold 7


/home/unmesh/anaconda3/envs/food-torch-env/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [17:08:11] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "categorical_feature" } are not used.

  warnings.warn(smsg, UserWarning)


ROC-AUC Score =  0.9998295618454334
3 of KFold 7


...

ROC-AUC Score =  0.9999795358349979
6 of KFold 7


/home/unmesh/anaconda3/envs/food-torch-env/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [17:14:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "categorical_feature" } are not used.

  warnings.warn(smsg, UserWarning)


ROC-AUC Score =  1.0
7 of KFold 7


/home/unmesh/anaconda3/envs/food-torch-env/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [17:14:51] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "categorical_feature" } are not used.

  warnings.warn(smsg, UserWarning)


ROC-AUC Score =  0.9999960904513183


In [76]:
print("Local CV Mean, STD ROC AUC SCORE for primary_close_flag_prediction model = ", np.mean(y1_cv_scores), np.std(y1_cv_scores))
print("Local CV Mean, STD ROC AUC SCORE for final_close_flag_prediction model = ", np.mean(y2_cv_scores), np.std(y2_cv_scores))

Local CV Mean, STD ROC AUC SCORE for primary_close_flag_prediction model =  0.9998345281151853 3.871346787194749e-05
Local CV Mean, STD ROC AUC SCORE for final_close_flag_prediction model =  0.9999919190809966 8.212895479923545e-06


In [77]:
y1_mean_preds = np.mean(y1_test_preds, axis=0)
y2_mean_preds = np.mean(y2_test_preds, axis = 0)

submn['primary_close_flag'] = y1_mean_preds
submn['final_close_flag'] = y2_mean_preds

submn

,primary_close_flag,final_close_flag
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
474284,0.0,0.0
474285,0.0,0.0
474286,0.0,0.0
474287,0.0,0.0


In [78]:
submn['primary_close_flag'] = np.where(submn['primary_close_flag'] >= 0.5, 1, 0)
submn['final_close_flag'] = np.where(submn['final_close_flag'] >= 0.5, 1, 0)


In [79]:
submn

,primary_close_flag,final_close_flag
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
474284,0,0
474285,0,0
474286,0,0
474287,0,0


In [80]:
submn.to_csv('XGB_SKF_7.csv', index=False)